In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split 
torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
Housing_datapath = "/content/drive/MyDrive/DL-Data/Housing.csv"
housing = pd.DataFrame(pd.read_csv(Housing_datapath))
x_var = ['area','bedrooms', 'bathrooms','stories','parking']
y_var = ['price']
input_layer = 5
samples = housing.shape[0]

In [3]:
np.random.seed(0) 
houseData_train, houseData_valid = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = np.random.randint(1))

In [4]:
x_data_train = houseData_train[x_var]
y_data_train = houseData_train[y_var]
x_data_valid = houseData_valid[x_var]
y_data_valid = houseData_valid[y_var]

x_train = torch.tensor(x_data_train.values, dtype = torch.float32)
x_valid = torch.tensor(x_data_valid.values, dtype = torch.float32)
y_train = torch.tensor(y_data_train.values, dtype = torch.float32)
y_valid = torch.tensor(y_data_valid.values, dtype = torch.float32)

x_ntrain = torch.nn.functional.normalize(x_train, dim = 0)
x_nvalid = torch.nn.functional.normalize(x_valid, dim = 0)
y_ntrain = torch.nn.functional.normalize(y_train, dim = 0)
y_nvalid = torch.nn.functional.normalize(y_valid, dim = 0)

In [5]:
def training_loop(n_epochs,optimizer,model,loss_fn,x_train_data,x_valid_data,y_train_data,y_valid_data):
  for epoch in range(1, n_epochs + 1):
    x_p_train = model(x_train_data)
    loss_train = loss_fn(x_p_train, y_train_data)
    x_p_valid = model(x_valid_data)
    loss_val = loss_fn(x_p_valid,y_valid_data)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 1 or epoch % 20 == 0:
       print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")


In [6]:
from collections import OrderedDict

nn11_model = nn.Sequential(OrderedDict([
            ('hidden_linear', nn.Linear(input_layer,8)),
            ('hidden_activation', nn.Tanh()),
            ('output_linear', nn.Linear(8, 1))
]))
nn11_model

Sequential(
  (hidden_linear): Linear(in_features=5, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [7]:
for name, param in nn11_model.named_parameters():
    print(name, param.shape)

nn11_model.output_linear.bias

hidden_linear.weight torch.Size([8, 5])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


Parameter containing:
tensor([-0.0371], requires_grad=True)

In [8]:
optimizer = optim.SGD(nn11_model.parameters(), lr=1e-3) 
print(y_valid.dtype)
training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = nn11_model,
    loss_fn = nn.MSELoss(),
    x_train_data = x_ntrain,
    x_valid_data = x_nvalid, 
    y_train_data = y_ntrain,
    y_valid_data = y_nvalid)

torch.float32
Epoch 1, Training loss 0.0788, Validation loss 0.1025
Epoch 20, Training loss 0.0694, Validation loss 0.0918
Epoch 40, Training loss 0.0608, Validation loss 0.0819
Epoch 60, Training loss 0.0533, Validation loss 0.0732
Epoch 80, Training loss 0.0467, Validation loss 0.0655
Epoch 100, Training loss 0.0410, Validation loss 0.0588
Epoch 120, Training loss 0.0360, Validation loss 0.0528
Epoch 140, Training loss 0.0316, Validation loss 0.0475
Epoch 160, Training loss 0.0277, Validation loss 0.0428
Epoch 180, Training loss 0.0244, Validation loss 0.0386
Epoch 200, Training loss 0.0214, Validation loss 0.0349
